In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [26]:
world_cup = pd.read_csv('World Cup 2019 Dataset.csv',encoding="windows-1252",sep = ',')
results = pd.read_csv('results.csv', encoding="windows-1252",sep = ',')
ranking = pd.read_csv('icc_rankings.csv',encoding="windows-1252") 
fixtures = pd.read_csv('fixtures.csv',encoding="windows-1252")

In [27]:
ranking.head()

,Position,Team,Points
0,1,Pakistan,270
1,2,Australia,269
2,3,England,265
3,4,South Africa,266
4,5,India,267


In [28]:
world_cup.head()

,Team,Group,Previous \r\nappearances,Previous \r\ntitles,Previous\r\n finals,Previous\r\n semifinals,Current \r rank,RunnersUp
0,England,A,6,1,2,0,3,1
1,South Africa,A,6,0,0,2,4,0
2,West Indies,A,6,2,2,2,10,0
3,Pakistan,A,6,1,2,2,1,1
4,New Zealand,A,6,0,0,2,6,0


In [29]:
results.head()

,date,Team_1,Team_2,Winner,Margin,Ground
0,4-Jan-10,Bangladesh,Sri Lanka,Sri Lanka,7 wickets,Dhaka
1,5-Jan-10,India,Sri Lanka,Sri Lanka,5 wickets,Dhaka
2,7-Jan-10,Bangladesh,India,India,6 wickets,Dhaka
3,8-Jan-10,Bangladesh,Sri Lanka,Sri Lanka,9 wickets,Dhaka
4,10-Jan-10,India,Sri Lanka,India,8 wickets,Dhaka


In [30]:
fixtures.head()

,Team_1,Team_2,Ground,Winner
0,Sri Lanka,Ireland,"Kardinia Park, South Geelong",NaN
1,Papua New Guinea,Oman,"Kardinia Park, South Geelong",NaN
2,Bangladesh,Namibia,"Bellerive Oval, Tasmania",NaN
3,Netherlands,Scotland,"Bellerive Oval, Tasmania",NaN
4,Ireland,Oman,"Kardinia Park, South Geelong",NaN


In [31]:
df = results[(results['Team_1'] == 'India') | (results['Team_2'] == 'India')]
india = df.iloc[:]
india.head()

,date,Team_1,Team_2,Winner,Margin,Ground
1,5-Jan-10,India,Sri Lanka,Sri Lanka,5 wickets,Dhaka
2,7-Jan-10,Bangladesh,India,India,6 wickets,Dhaka
4,10-Jan-10,India,Sri Lanka,India,8 wickets,Dhaka
5,11-Jan-10,Bangladesh,India,India,6 wickets,Dhaka
6,13-Jan-10,India,Sri Lanka,Sri Lanka,4 wickets,Dhaka


In [32]:
year = []
for row in india['Team_1']:
    year.append(int(row[7:]))
india ['match_year']= year
india_2010 = india[india.match_year >= 10]
india_2010.count()

ValueError: invalid literal for int() with base 10: ''

In [33]:
worldcup_teams = ['England', ' South Africa', '', 'West Indies', 
            'Pakistan', 'New Zealand', 'Sri Lanka', 'Afghanistan', 
            'Australia', 'Bangladesh', 'India','Papua New Guinea','Ireland','Oman','Namibia','Scotland','Netherlands']
df_teams_1 = results[results['Team_1'].isin(worldcup_teams)]
df_teams_2 = results[results['Team_2'].isin(worldcup_teams)]
df_teams = pd.concat((df_teams_1, df_teams_2))
df_teams.drop_duplicates()
df_teams.count()

date      1611
Team_1    1611
Team_2    1611
Winner    1611
Margin    1523
Ground    1611
dtype: int64

In [36]:
df_teams_2010 = df_teams.drop(['Ground','Margin','date'], axis=1)
df_teams_2010.head()

,Team_1,Team_2,Winner
0,Bangladesh,Sri Lanka,Sri Lanka
1,India,Sri Lanka,Sri Lanka
2,Bangladesh,India,India
3,Bangladesh,Sri Lanka,Sri Lanka
4,India,Sri Lanka,India


In [37]:
df_teams_2010 = df_teams_2010.reset_index(drop=True)
df_teams_2010.loc[df_teams_2010.Winner == df_teams_2010.Team_1,'winning_team']=1
df_teams_2010.loc[df_teams_2010.Winner == df_teams_2010.Team_2, 'winning_team']=2
df_teams_2010 = df_teams_2010.drop(['winning_team'], axis=1)

df_teams_2010.head()

,Team_1,Team_2,Winner
0,Bangladesh,Sri Lanka,Sri Lanka
1,India,Sri Lanka,Sri Lanka
2,Bangladesh,India,India
3,Bangladesh,Sri Lanka,Sri Lanka
4,India,Sri Lanka,India


In [38]:
final = pd.get_dummies(df_teams_2010, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

X = final.drop(['Winner'], axis=1)
y = final["Winner"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [39]:
rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=0)
rf.fit(X_train, y_train) 


score = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)


print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.712
Test set accuracy:  0.649


In [40]:
#ranking = pd.read_csv('icc_rankings1.csv',encoding="windows-1252") 
#fixtures = pd.read_csv('fixtures2.csv',encoding="windows-1252")
pred_set = []

In [41]:
fixtures.insert(1, 'first_position', fixtures['Team_1'].map(ranking.set_index('Team')['Position']))
fixtures.insert(2, 'second_position', fixtures['Team_2'].map(ranking.set_index('Team')['Position']))

fixtures = fixtures.iloc[:45, :]
fixtures.tail()

,Team_1,first_position,second_position,Team_2,Ground,Winner
40,South Africa,NaN,NaN,Ireland,"Sydney Cricket Ground, Sydney",NaN
41,India,NaN,NaN,Afghanistan,"Sydney Cricket Ground, Sydney",NaN
42,To be announced,NaN,NaN,To be announced,"Sydney Cricket Ground, Sydney (7.00pm)",NaN
43,To be announced,NaN,NaN,To be announced,"Adelaide Oval, Adelaide (7.00pm)",NaN
44,To be announced,NaN,NaN,To be announced,"Melbourne Cricket Ground, Melbourne",NaN


In [42]:
for index, row in fixtures.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'Team_1': row['Team_1'], 'Team_2': row['Team_2'], 'winning_team': None})
    else:
        pred_set.append({'Team_1': row['Team_2'], 'Team_2': row['Team_1'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set
pred_set.head()

,Team_1,Team_2,winning_team
0,Ireland,Sri Lanka,None
1,Oman,Papua New Guinea,None
2,Namibia,Bangladesh,None
3,Scotland,Netherlands,None
4,Oman,Ireland,None


In [43]:
pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]


pred_set = pred_set.drop(['Winner'], axis=1)
pred_set.head()

,Team_1_Afghanistan,Team_1_Australia,Team_1_Bangladesh,Team_1_Canada,Team_1_England,Team_1_Hong Kong,Team_1_India,Team_1_Ireland,Team_1_Kenya,Team_1_Netherlands,...,Team_2_Kenya,Team_2_Netherlands,Team_2_New Zealand,Team_2_Pakistan,Team_2_Scotland,Team_2_South Africa,Team_2_Sri Lanka,Team_2_U.A.E.,Team_2_West Indies,Team_2_Zimbabwe
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
predictions = rf.predict(pred_set)
for i in range(fixtures.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 1:
        print("Winner: " + backup_pred_set.iloc[i, 1])
    
    else:
        print("Winner: " + backup_pred_set.iloc[i, 0])
    print("")

Sri Lanka and Ireland
Winner: Ireland

Papua New Guinea and Oman
Winner: Oman

Bangladesh and  Namibia
Winner:  Namibia

Netherlands and Scotland 
Winner: Scotland 

Ireland and   Oman
Winner:   Oman

Sri Lanka and  Papua New Guinea 
Winner:  Papua New Guinea 

Namibia and Scotland
Winner: Scotland

Bangladesh and Netherlands
Winner: Netherlands

Papua New Guinea and  Ireland
Winner:  Ireland

Sri Lanka and Oman
Winner: Oman

Netherlands and  Namibia
Winner:  Namibia

Bangladesh and  Scotland
Winner:  Scotland

Australia and Pakistan 
Winner: Pakistan 

India and South Africa
Winner: South Africa

Srilanka and  Netherlands
Winner:  Netherlands

New Zealand  and  West Indies
Winner:  West Indies

Afghanistan   and  Bangladesh
Winner:  Bangladesh

England  and  Ireland
Winner:  Ireland

New Zealand   and  Netherlands
Winner:  Netherlands

Afghanistan   and Ireland
Winner: Ireland

Australia  and  West Indies
Winner:  West Indies

Pakistan  and  Srilanka
Winner:  Srilanka

India and Bangl

In [45]:
# List of tuples before 
semi = [('New Zealand', 'India'),
            ('England', 'South Africa')]

In [46]:
def clean_and_predict(matches, ranking, final, logreg):

    # Initialization of auxiliary list for data cleaning
    positions = []

    # Loop to retrieve each team's position according to ICC ranking
    for match in matches:
        positions.append(ranking.loc[ranking['Team'] == match[0],'Position'].iloc[0])
        positions.append(ranking.loc[ranking['Team'] == match[1],'Position'].iloc[0])
    
    # Creating the DataFrame for prediction
    pred_set = []

    # Initializing iterators for while loop
    i = 0
    j = 0

    # 'i' will be the iterator for the 'positions' list, and 'j' for the list of matches (list of tuples)
    while i < len(positions):
        dict1 = {}

        # If position of first team is better then this team will be the 'Team_1' team, and vice-versa
        if positions[i] < positions[i + 1]:
            dict1.update({'Team_1': matches[j][0], 'Team_2': matches[j][1]})
        else:
            dict1.update({'Team_1': matches[j][1], 'Team_2': matches[j][0]})

        # Append updated dictionary to the list, that will later be converted into a DataFrame
        pred_set.append(dict1)
        i += 2
        j += 1
        
        # Convert list into DataFrame
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set

    # Get dummy variables and drop winning_team column
    pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

    # Add missing columns compared to the model's training dataset
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    pred_set = pred_set.drop(['Winner'], axis=1)

    # Predict!
    predictions = logreg.predict(pred_set)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 1:
            print("Winner: " + backup_pred_set.iloc[i, 1])
        else:
            print("Winner: " + backup_pred_set.iloc[i, 0])
        print("")

In [47]:
clean_and_predict(semi, ranking, final, rf)

IndexError: single positional indexer is out-of-bounds